In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import os
import math
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt

In [2]:
coords_path = "/external/rachelwong/Documents/HMDB51/Videos_Pose/Coords/"

In [4]:
dataset = CoordsDataset(coords_path, frames_per_segment=50, 
                        frame_stride=5, max_people=6,
                        start_percent_skip=.1, end_percent_skip=.1)
print(len(dataset))

loaders = get_train_val_test_loaders(dataset, batch_size=64)
(trainloader, valloader, testloader) = loaders
print(len(trainloader))

for i, (data, labels) in enumerate(trainloader):
    print(i, data.shape, labels.shape)

8212
103
0 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
1 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
2 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
3 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
4 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
5 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
6 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
7 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
8 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
9 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
10 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
11 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
12 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
13 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
14 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
15 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
16 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
17 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
18 torch.Size([64, 10, 6, 137, 3]) torch.Size([64])
19 torch.Size

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=3):
        super().__init__()

        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.transpose(0,1)
        output, hidden = self.rnn(x)
        return self.fc(output[-1])

def binary_accuracy(preds, y):
    _, max_idxs = preds.max(1)
    correct = (max_idxs == y).float()
    return correct.sum()/len(correct)

def train_epoch(model, loader, optimizer):
    loss_epoch, acc_epoch = 0, 0
    for data, labels in loader:
        data, labels = data.cuda(), labels.cuda()
        output = model(data)
        loss = F.cross_entropy(output, labels)
        acc = binary_accuracy(output, labels)

        loss.backward()
        optimizer.step()

        loss_epoch += loss.item()
        acc_epoch += acc.item()

    return loss_epoch / len(loader), acc_epoch / len(loader)

def test_nn(data_dir):
    dataset = CoordsDataset(data_dir)
    trainloader, valloader, testloader = get_train_val_test(dataset)

    model = RNN(75, 100, 2).cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    for epoch in range(40):
        loss, acc = train_epoch(model, trainloader, optimizer)
        print(F"Epoch: {epoch+1} Loss: {loss} \tAcc: {acc}")